In [1]:
from fastembed import TextEmbedding

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from qdrant_client import QdrantClient, models

In [3]:
qd_client = QdrantClient("http://localhost:6333") #connecting to local Qdrant instance

In [4]:
EMBEDDING_DIMENSIONALITY = 512
model_handle = "jinaai/jina-embeddings-v2-small-en"
collection_name = "zoomcamp-faq"

In [5]:
embedder = TextEmbedding(model_name = model_handle)

In [6]:
q = 'I just discovered the course. Can I join now?'

In [7]:
embed_query = list(embedder.embed(q))

In [8]:
import numpy as np

## Q1. Embedding the query

In [9]:
embed_array = np.array(embed_query)
print(embed_array.shape)
print(embed_array.min())

(1, 512)
-0.11726373885183883


## Q2. Cosine similarity with another vector

In [10]:
np.linalg.norm(embed_array)

np.float64(1.0)

In [11]:
embed_query[0].dot(embed_query[0])

np.float64(1.0000000000000002)

In [12]:
doc = 'Can I still join the course after the start date?'

In [13]:
embed_doc = list(embedder.embed(doc))

In [14]:
embed_query[0].dot(embed_doc[0])

np.float64(0.9008528895674548)

## Q3. Ranking by Cosine

In [15]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

In [16]:
text_doc = [doc['text'] for doc in documents]
text_doc

["Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
 "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'You can start by instal

In [17]:
embed_text = list(embedder.embed(text_doc))
embed_text

[array([-2.49525006e-02, -3.96454034e-02, -4.37672429e-03,  2.95830380e-02,
        -1.20300744e-02, -6.93249608e-03, -7.65465072e-05,  6.15121401e-02,
        -2.85105823e-02, -6.70960944e-02,  6.53191995e-03, -5.09575722e-02,
        -5.01244174e-02, -5.83276165e-02,  9.54578951e-02, -5.41039925e-02,
        -3.20385860e-02, -4.69560323e-02, -6.16868024e-02, -9.03934389e-02,
        -3.35349282e-02, -2.90533971e-02,  4.44972092e-02, -7.00045540e-02,
         4.27595264e-02, -7.01989178e-02, -8.68037276e-02,  8.14205881e-02,
         7.21879907e-02,  1.89472918e-02, -5.95576718e-02,  1.09493119e-02,
        -1.80511540e-02,  4.60817619e-02, -2.97588624e-02,  3.45792057e-02,
         4.05221611e-02,  1.75949846e-02, -2.77041533e-02, -1.67211727e-02,
        -5.02859986e-02,  1.34971538e-02,  9.17410131e-02,  8.32858616e-03,
        -7.49552827e-02,  2.35092282e-03, -4.66536576e-02,  1.29560582e-02,
         2.57738903e-02,  5.87156475e-03, -4.75062527e-02,  1.41338343e-02,
        -1.0

In [18]:
V = np.array(embed_text)
result = V.dot(embed_query[0])
result

array([0.76296847, 0.81823782, 0.80853974, 0.7133079 , 0.73044992])

In [19]:
print(q)
print(result.argmax())
print(documents[result.argmax()])

I just discovered the course. Can I join now?
1
{'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.', 'section': 'General course-related questions', 'question': 'Course - Can I follow the course after it finishes?', 'course': 'data-engineering-zoomcamp'}


## Q4. Ranking by cosine, version two

In [20]:
full_text = [doc['question'] + ' ' + doc['text'] for doc in documents]
full_text

["Course - Can I still join the course after the start date? Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
 'Course - Can I follow the course after it finishes? Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
 "Course - When will the course start? The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin t

In [21]:
embed_doc = list(embedder.embed(full_text))
embed_doc

[array([-3.95874826e-02, -6.90406894e-02,  1.14661797e-02,  2.62603988e-02,
        -2.47341027e-02, -2.47207056e-02,  3.43971205e-03,  4.27859250e-02,
        -2.41629394e-02, -2.61992357e-02, -1.49214508e-02, -7.88178399e-02,
        -2.47400895e-02, -7.74014517e-02,  7.68642757e-02, -4.65683749e-02,
        -3.08454460e-02, -4.45657217e-02, -4.48141420e-02, -6.06556355e-02,
        -3.67113226e-02, -1.80009404e-02,  3.19522744e-02, -6.80573414e-02,
         5.93400861e-02, -7.10708989e-02, -8.38071366e-02,  7.13435390e-02,
         7.23555512e-02,  2.43478195e-02, -5.51024677e-02,  2.94591679e-02,
        -1.83082436e-02,  2.56099754e-02, -3.17286899e-02,  2.76751097e-02,
         4.96358524e-02,  1.38839466e-02, -2.01003038e-02, -2.82512659e-03,
        -2.93719094e-02,  2.82632724e-02,  1.00648435e-01,  2.89436669e-03,
        -7.16561258e-02,  7.67853418e-04, -8.02424389e-02,  1.12407843e-02,
         2.55503821e-02,  1.53881720e-02, -3.34380302e-02,  2.85950294e-02,
        -7.9

In [22]:
V2 = np.array(embed_doc)
result_2 = V2.dot(embed_query[0])
result_2

array([0.85145432, 0.84365942, 0.8408287 , 0.7755158 , 0.80860078])

In [23]:
result_2.argmax()

np.int64(0)

In [24]:
print(q)
print(result_2.argmax())
print(full_text[result_2.argmax()])

I just discovered the course. Can I join now?
0
Course - Can I still join the course after the start date? Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.


The result now is different from Q3 because the question was added to the context

## Q5. Select the embedding model
Now let's select a smaller embedding model. What's the smallest dimensionality for models in fastembed?

In [25]:
import pandas as pd

In [29]:
TextEmbedding.list_supported_models()[0]

{'model': 'BAAI/bge-base-en',
 'sources': {'hf': 'Qdrant/fast-bge-base-en',
  'url': 'https://storage.googleapis.com/qdrant-fastembed/fast-bge-base-en.tar.gz',
  '_deprecated_tar_struct': True},
 'model_file': 'model_optimized.onnx',
 'description': 'Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: necessary, 2023 year.',
 'license': 'mit',
 'size_in_GB': 0.42,
 'additional_files': [],
 'dim': 768,
 'tasks': {}}

In [41]:

model_df = pd.DataFrame([
    {
        'model' : m['model'],
        'description' : m['description'],
        'size_in_GB' : m['size_in_GB'],
        'dim' : m['dim']
    } for m in TextEmbedding.list_supported_models()
])

model_df.sort_values('dim').head()

,model,description,size_in_GB,dim
3,BAAI/bge-small-en,"Text embeddings, Unimodal (text), English, 512...",0.130,384
4,BAAI/bge-small-en-v1.5,"Text embeddings, Unimodal (text), English, 512...",0.067,384
7,snowflake/snowflake-arctic-embed-xs,"Text embeddings, Unimodal (text), English, 512...",0.090,384
8,snowflake/snowflake-arctic-embed-s,"Text embeddings, Unimodal (text), English, 512...",0.130,384
14,sentence-transformers/all-MiniLM-L6-v2,"Text embeddings, Unimodal (text), English, 256...",0.090,384


## Q6. Indexing with qdrant

In [42]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

In [44]:
documents_raw[0]

{'course': 'data-engineering-zoomcamp',
 'documents': [{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
   'section': 'General course-related questions',
   'question': 'Course - When will the course start?'},
  {'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites',
   'section': 'General course-related questions',
   'question': 'Course - What are the prerequisites for this course?'},
  {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the fin

In [52]:
documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

documents[0]

{'text': 'Machine Learning Zoomcamp FAQ\nThe purpose of this document is to capture frequently asked technical questions.\nWe did this for our data engineering course and it worked quite well. Check this document for inspiration on how to structure your questions and answers:\nData Engineering Zoomcamp FAQ\nIn the course GitHub repository there’s a link. Here it is: https://airtable.com/shryxwLd0COOEaqXo\nwork',
 'section': 'General course-related questions',
 'question': 'How do I sign up?',
 'course': 'machine-learning-zoomcamp'}